<a href="https://colab.research.google.com/github/ovokojo/fintequity/blob/main/How%20to%20Analyze%20Stocks%20with%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook aims to explore possibilities of AI in investment analysis. It uses various APIs to pull data and perform analyses. To ensure a smooth experience, please follow the setup instructions below.

**Pre-requisites**

Before running this notebook, you will need access to the following APIs:


*   **SEC-API** for accessing financial statements and other disclosures from the U.S. Securities and Exchange Commission. Get a free API key at: https://sec-api.io/

*   **OpenAI API** for utilizing advanced AI models for data analysis, summaries, and more. Get an API key at: https://platform.openai.com/



**Getting Started**

Configure API Keys: Store your SEC-API and OpenAI API keys in a secure location. This notebook uses Colab Secrets.

Run the notebook: Execute the cells in order, making sure to input your API keys or modify the code as neccessary to access them.



In [11]:
!pip install -q sec-api
!pip install openai

from google.colab import userdata
import openai
from openai import OpenAI
import requests

client = OpenAI(
    api_key=userdata.get("OAI_KEY"),
)
SEC_KEY = userdata.get("SEC_KEY")

In [12]:
# Query API - Search and filter all 18+ million filings and exhibits published on SEC EDGAR since 1993
def getLatest10K(ticker: str):
    """
    Retrieves the most recent Form 10-K filing for a given stock ticker symbol.

    Args:
    - ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.

    Returns:
    - dict: A dictionary containing the JSON response with the latest Form 10-K filing
            information if the request is successful (HTTP status code 200).
    - int: The HTTP status code of the response if the request is unsuccessful.
    """

    endpoint = f"https://api.sec-api.io?token={SEC_KEY}"
    params = {
        "query": {
          "query_string": {
            "query":  f"ticker:{ticker} AND formType:\"10-K\""
          }
        },
        "from": "0",
        "size": "1",
        "sort": [{ "filedAt": { "order": "desc" }}]
    }
    response = requests.post(endpoint, json=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        return response.status_code

# Example
# latest10K = getLatest10K('TSLA')

In [13]:
# Extractor API - Extract any text section from 10-Q, 10-K, and 8-K SEC filings.
def getLatest10kSection(ticker: str, section: str):
    """
    Retrieves a specified section from the latest 10-K filing of any stock using the SEC Query API & the extractor API

    Args:
        ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.
        section (str): Specific section of the filing to retrieve (e.g., "1A" for the Risk Factors section).
        10-K supported section codes: 1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15

    Returns:
        text or int: The extracted content in JSON format if request is successful; otherwise, the HTTP status code.
    """

    latest10K = getLatest10K(ticker)
    url_10k = latest10K['filings'][0]['linkToFilingDetails']

    endpoint = "https://api.sec-api.io/extractor"

    params = {
        "token": SEC_KEY,
        "url": url_10k,
        "item": section,
        "type": "text"
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with code: {response.status_code}")
        return response.status_code

# Example
# getLatest10kSection("WDAY", "1A")

In [18]:
# Next steps: analyze & summarize section with AI
def askAssistant(ticker: str, section: str, prompt: str):
  data = getLatest10kSection(ticker, section)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"{prompt}. Use {section} of {ticker} 10-K here: {data}",
          }
      ],
      model="gpt-4-turbo-preview",
  )
  response = chat_completion.choices[0].message.content
  print(response)

In [20]:
# Example use case: get executive summaries of multiple companies.
tickers = ["ABNB","UBER","LYFT"]

summaries = []

for ticker in tickers:
  askAssistant(ticker, "1", "Give me a summary of the business model")
  print("\n")

Airbnb operates within a community-based model focused on connection and belonging, originating from 2007 when two hosts in San Francisco welcomed three guests. The platform has since grown to over 5 million hosts globally, with over 1.5 billion guest arrivals. Airbnb caters to five key stakeholders: hosts, guests, employees, shareholders, and communities, aiming for mutual success to nurture its business health.

The business model highlights adaptability and innovation as its core strengths, allowing it to offer a wide variety of stay and experience types across almost every global location. Despite varying economic climates, Airbnb has maintained appeal by offering value to guests and income opportunities for hosts. The company emphasizes growth strategies such as mainstreaming hosting, engaging the guest community, perfecting core services, and expanding its global network. Investing in hosting support, product improvements, international markets, and adapting to new travel behavio